In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 4dfe57c9-d279-4f15-9130-7b98f0646608, 3, Finished, Available, Finished)

Write this into copilot
```
guess data type when loading
```

In [2]:
#### ATTENTION: AI-generated code can include errors or operations you didn't intend. Review the code in this cell carefully before running it.

# Load CSV data into a Spark DataFrame
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("Files/raw/churn.csv")

# Display the DataFrame to infer schema and inspect data
display(df)

StatementMeta(, 4dfe57c9-d279-4f15-9130-7b98f0646608, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 427da532-79b4-495d-be66-c3af3ebdd30f)

In [3]:
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.sql import types as T

def clean_data(df):
    # Drop duplicate rows in columns: 'RowNumber', 'CustomerId'
    df = df.dropDuplicates(['RowNumber', 'CustomerId'])
    # Drop rows with missing data across all columns
    df = df.dropna()
    # Drop columns: 'RowNumber', 'CustomerId', 'Surname'
    df = df.drop('RowNumber', 'CustomerId', 'Surname')
    return df

df_clean = clean_data(df)
display(df_clean)

StatementMeta(, 4dfe57c9-d279-4f15-9130-7b98f0646608, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7c0e81fe-a3d5-4f9c-a2ec-7a356af025a7)

Write this in data wrangler :
```
  NewTenure = Tenure / Age
```

```
 qcut on CreditScore into 6 quantile bins (NewCreditsScore) , import necessary lib
```

```
 qcut on Age into 8 quantile bins (NewAgeScore), import necessary lib
```

```
 apply qcut to balance with rank first and 5 bins (NewBalanceScore)
```

```
 qcut on EstimatedSalary into 10 bins (NewEstSalaryScore), import necessary lib
```

In [4]:
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.sql.functions import col
from pyspark.sql.functions import col, ntile
from pyspark.sql.functions import ntile
from pyspark.sql.functions import percent_rank
from pyspark.sql.window import Window

def clean_data(df_clean):
    # LLM translation result for custom operation
    df_clean = df_clean.withColumn('NewTenure', col('Tenure') / col('Age'))
    # LLM translation result for custom operation
    # Create a window specification
    windowSpec = Window.orderBy('CreditScore')
    # Add a new column with quantile bins
    df_clean = df_clean.withColumn('NewCreditScore', ntile(6).over(windowSpec) - 1)
    # LLM translation result for custom operation
    # Create a window specification for quantile binning
    window_spec = Window.orderBy(col('Age'))
    # Add a new column 'NewAgeScore' with quantile bins
    df_clean = df_clean.withColumn('NewAgeScore', ntile(8).over(window_spec) - 1)
    # LLM translation result for custom operation
    windowSpec = Window.orderBy("Balance")
    df_clean = df_clean.withColumn("rank", percent_rank().over(windowSpec))
    df_clean = df_clean.withColumn("NewBalanceScore", (df_clean["rank"] * 5).cast("int"))
    # LLM translation result for custom operation
    windowSpec = Window.orderBy("EstimatedSalary")
    df_clean = df_clean.withColumn("NewEstSalaryScore", ntile(10).over(windowSpec) - 1)
    return df_clean

df_clean_1 = clean_data(df_clean)
display(df_clean_1)

StatementMeta(, 4dfe57c9-d279-4f15-9130-7b98f0646608, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a7342d72-248c-4528-bbc3-302b4b4e9d0c)

In [5]:
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array
from pyspark.sql import functions as F

def clean_data(df_clean_1):
    # One-hot encode columns: 'Geography', 'Gender'
    # ⚠️ This was generated to match the original pandas logic but may have performance issues.
    def one_hot_encode_col(df, key):
        insert_idx = df.columns.index(key)
        indexer = StringIndexer(inputCol=key, outputCol='%s_numeric' % str(key), handleInvalid='keep')
        indexer_fitted = indexer.fit(df)
        df_indexed = indexer_fitted.transform(df)
        encoder = OneHotEncoder(inputCols=['%s_numeric' % str(key)], outputCols=['%s_onehot' % str(key)], dropLast=False)
        df_onehot = encoder.fit(df_indexed).transform(df_indexed)
        df_col_onehot = df_onehot.select('*', vector_to_array('%s_onehot' % str(key)).alias('%s_col_onehot' % str(key)))
        labels = sorted(indexer_fitted.labels)
        cols_expanded = [(F.col('%s_col_onehot' % str(key))[i].alias('%s_%s' % (str(key), labels[i]))) for i in range(len(labels))]
        df = df_col_onehot.select(*df.columns[:insert_idx], *cols_expanded, *df.columns[insert_idx+1:])
        return df
    df_clean_1 = one_hot_encode_col(df_clean_1, 'Geography')
    df_clean_1 = one_hot_encode_col(df_clean_1, 'Gender')
    return df_clean_1

df_clean_2 = clean_data(df_clean_1)
display(df_clean_2)

StatementMeta(, 4dfe57c9-d279-4f15-9130-7b98f0646608, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a98547e2-d882-4829-be34-e1d451e26a0b)

Write this into copilot
```
save df_clean_2 table into the default lakehouse in a delta table named "df_clean"
```

In [ ]:
#### ATTENTION: AI-generated code can include errors or operations you didn't intend. Review the code in this cell carefully before running it.

# Export df_clean_2 to a Delta table named "df_clean" in the default lakehouse
df_clean_2.write.format("delta").saveAsTable("df_clean")

# If you want to overwrite the table (WARNING: this will replace existing table)
# df_clean_2.write.mode("overwrite").format("delta").saveAsTable("df_clean")

StatementMeta(, 4dfe57c9-d279-4f15-9130-7b98f0646608, 8, Finished, Available, Finished)

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `churnlakehousenocode`.`df_clean` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.